In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
df= pd.read_csv(r"C:\Users\amnaj\OneDrive\Desktop\zomoto\cleaned_restaurant_data.csv")
print(df)

             restaurant_name     restaurant_type  rate_(out_of_5)  \
0                feeltheroll         quick bites              3.0   
1                  l-81 cafe         quick bites              4.0   
2                     refuel                cafe              4.0   
3            biryani central       casual dining              3.0   
4                    the bbq       casual dining              3.0   
...                      ...                 ...              ...   
6382                   zoeys                cafe              4.0   
6383  zoroy luxury chocolate      dessert parlor              4.0   
6384        zus doner kebaps  takeaway, delivery              4.0   
6385                   zyara       casual dining              4.0   
6386                  zyksha          food truck              3.0   

      num_of_ratings  avg_cost_(two_people)  online_order  table_booking  \
0                  7                    200             0              0   
1                 4

In [3]:
df.columns

Index(['restaurant_name', 'restaurant_type', 'rate_(out_of_5)',
       'num_of_ratings', 'avg_cost_(two_people)', 'online_order',
       'table_booking', 'cuisines_type', 'area', 'local_address',
       'num_cuisines'],
      dtype='object')

In [4]:
df.dtypes

restaurant_name           object
restaurant_type           object
rate_(out_of_5)          float64
num_of_ratings             int64
avg_cost_(two_people)      int64
online_order               int64
table_booking              int64
cuisines_type             object
area                      object
local_address             object
num_cuisines               int64
dtype: object

In [5]:
df['rate_(out_of_5)'] = np.round(df['rate_(out_of_5)']).astype('Int64')
df['avg_cost_(two_people)'] = np.round(df['avg_cost_(two_people)']).astype('Int64')
print(df['rate_(out_of_5)'])
print(df['avg_cost_(two_people)'])

0       3
1       4
2       4
3       3
4       3
       ..
6382    4
6383    4
6384    4
6385    4
6386    3
Name: rate_(out_of_5), Length: 6387, dtype: Int64
0       200
1       400
2       400
3       550
4       700
       ... 
6382    600
6383    250
6384    350
6385    650
6386    200
Name: avg_cost_(two_people), Length: 6387, dtype: Int64


In [6]:
df.dtypes

restaurant_name          object
restaurant_type          object
rate_(out_of_5)           Int64
num_of_ratings            int64
avg_cost_(two_people)     Int64
online_order              int64
table_booking             int64
cuisines_type            object
area                     object
local_address            object
num_cuisines              int64
dtype: object

In [9]:
X = df[['restaurant_type', 'rate_(out_of_5)', 'num_of_ratings',
        'online_order', 'table_booking', 'cuisines_type',
        'area', 'num_cuisines']].copy()
y = df['avg_cost_(two_people)']


In [15]:
# clean both columns
df['online_order'] = df['online_order'].fillna(0).astype(int)
df['table_booking'] = df['table_booking'].fillna(0).astype(int)

# now safe to use in model
X = df[['restaurant_type', 'rate_(out_of_5)', 'num_of_ratings',
        'online_order', 'table_booking', 'cuisines_type',
        'area', 'num_cuisines']].copy()


In [18]:
df.dtypes

restaurant_name          object
restaurant_type          object
rate_(out_of_5)           Int64
num_of_ratings            int64
avg_cost_(two_people)     Int64
online_order              int64
table_booking             int64
cuisines_type            object
area                     object
local_address            object
num_cuisines              int64
dtype: object

In [20]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def clean_restaurant_data(df):
    """
    Cleans and encodes restaurant dataset for regression or classification.
    """
    df = df.copy()  # avoid modifying original
    
    # 1️⃣ Handle missing values
    df = df.fillna({
        'online_order': 0,
        'table_booking': 0,
        'rate_(out_of_5)': df['rate_(out_of_5)'].median(),
        'avg_cost_(two_people)': df['avg_cost_(two_people)'].median(),
        'num_of_ratings': df['num_of_ratings'].median(),
        'num_cuisines': df['num_cuisines'].median()
    })
    
    # 2️⃣ Convert binary 0/1 columns properly
    for col in ['online_order', 'table_booking']:
        if col in df.columns:
            df[col] = df[col].astype(int)
    
    # 3️⃣ Drop unnecessary columns
    drop_cols = ['restaurant_name', 'local_address']
    df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')
    
    # 4️⃣ Encode categorical columns using LabelEncoder
    label_encoders = {}
    cat_cols = df.select_dtypes(include='object').columns
    
    for col in cat_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le  # save encoder for decoding later
    
    print("✅ Data cleaned and encoded successfully!")
    print(f"Final columns: {list(df.columns)}")
    return df, label_encoders


In [21]:
clean_df, encoders = clean_restaurant_data(df)


✅ Data cleaned and encoded successfully!
Final columns: ['restaurant_type', 'rate_(out_of_5)', 'num_of_ratings', 'avg_cost_(two_people)', 'online_order', 'table_booking', 'cuisines_type', 'area', 'num_cuisines']


In [23]:
# ✅ Define X (features) and y (target)
X = clean_df.drop('avg_cost_(two_people)', axis=1)
y = clean_df['avg_cost_(two_people)']

print(X.shape, y.shape)


(6387, 8) (6387,)


In [34]:
X = clean_df.drop('avg_cost_(two_people)', axis=1)
y = clean_df['avg_cost_(two_people)']

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
print("R² Score:", r2_score(y_test, model.predict(X_test)))
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

R² Score: 0.40401430752756495
Coefficients: [-5.07552235e+00 -2.35570743e+00  8.92193265e-02  8.69254211e+00
  2.27775017e+02  1.54538382e-02  7.39555435e-01  3.51385251e+01]
Intercept: 482.8272899172467


In [35]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R² Score: {r2:.4f}")
print(f"MSE: {mse:.2f}")


R² Score: 0.4040
MSE: 28015.80


In [36]:
import numpy as np

# Example: user input
restaurant_type = 3
rate = 4
num_of_ratings = 120
online_order = 1
table_booking = 0
cuisines_type = 5
area = 2
num_cuisines = 3

# Combine all features
features = np.array([[restaurant_type, rate, num_of_ratings, online_order,
                      table_booking, cuisines_type, area, num_cuisines]])

# Predict cost
predicted_cost = model.predict(features)
print("Predicted average cost for two people:", round(predicted_cost[0], 2))


Predicted average cost for two people: 584.55


C:\Users\amnaj\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [38]:
import numpy as np

# Take inputs from user
restaurant_type_input = input("Enter restaurant type: ")
rate = float(input("Enter rating (out of 5): "))
num_of_ratings = int(input("Enter number of ratings: "))
online_order = int(input("Online order available? (1 = Yes, 0 = No): "))
table_booking = int(input("Table booking available? (1 = Yes, 0 = No): "))
cuisines_type_input = input("Enter cuisine type: ")
area_input = input("Enter area: ")
num_cuisines = int(input("Enter number of cuisines: "))

# Convert text inputs to encoded numeric form
restaurant_type = encoders['restaurant_type'].transform([restaurant_type_input])[0]
cuisines_type = encoders['cuisines_type'].transform([cuisines_type_input])[0]
area = encoders['area'].transform([area_input])[0]

# Prepare features
features = np.array([[restaurant_type, rate, num_of_ratings, online_order,
                      table_booking, cuisines_type, area, num_cuisines]])

# Predict cost
predicted_cost = model.predict(features)
print("\n💰 Predicted average cost for two people:", round(predicted_cost[0], 2))


Enter restaurant type:  quick bites
Enter rating (out of 5):  3
Enter number of ratings:  2
Online order available? (1 = Yes, 0 = No):  1
Table booking available? (1 = Yes, 0 = No):  0
Enter cuisine type:  fast food
Enter area:  Electronic City
Enter number of cuisines:  1



💰 Predicted average cost for two people: 279.57


C:\Users\amnaj\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [39]:
print("Available Restaurant Types:")
print(list(encoders['restaurant_type'].classes_))

print("\nAvailable Cuisine Types:")
print(list(encoders['cuisines_type'].classes_))

print("\nAvailable Areas:")
print(list(encoders['area'].classes_))


Available Restaurant Types:
['bakery', 'bakery, beverage shop', 'bakery, cafe', 'bakery, dessert parlor', 'bakery, quick bites', 'bar', 'bar, casual dining', 'bar, pub', 'beverage shop', 'beverage shop, cafe', 'beverage shop, dessert parlor', 'beverage shop, quick bites', 'bhojanalya', 'cafe', 'cafe, bakery', 'cafe, casual dining', 'cafe, dessert parlor', 'cafe, lounge', 'cafe, quick bites', 'casual dining', 'casual dining, bar', 'casual dining, cafe', 'casual dining, lounge', 'casual dining, microbrewery', 'casual dining, pub', 'casual dining, quick bites', 'casual dining, sweet shop', 'confectionery', 'delivery', 'dessert parlor', 'dessert parlor, bakery', 'dessert parlor, beverage shop', 'dessert parlor, cafe', 'dessert parlor, food court', 'dessert parlor, quick bites', 'dessert parlor, sweet shop', 'dhaba', 'fine dining', 'food court', 'food court, beverage shop', 'food court, casual dining', 'food court, dessert parlor', 'food court, quick bites', 'food truck', 'kiosk', 'lounge',

In [40]:
import numpy as np

def safe_encode(encoder, value):
    """Safely encode user input — use default if unseen."""
    if value in encoder.classes_:
        return encoder.transform([value])[0]
    else:
        print(f"⚠️ '{value}' not seen before. Using default value '{encoder.classes_[0]}' instead.")
        return encoder.transform([encoder.classes_[0]])[0]


# Take inputs from user
restaurant_type_input = input("Enter restaurant type: ")
rate = float(input("Enter rating (out of 5): "))
num_of_ratings = int(input("Enter number of ratings: "))
online_order = int(input("Online order available? (1 = Yes, 0 = No): "))
table_booking = int(input("Table booking available? (1 = Yes, 0 = No): "))
cuisines_type_input = input("Enter cuisine type: ")
area_input = input("Enter area: ")
num_cuisines = int(input("Enter number of cuisines: "))

# Encode safely
restaurant_type = safe_encode(encoders['restaurant_type'], restaurant_type_input)
cuisines_type = safe_encode(encoders['cuisines_type'], cuisines_type_input)
area = safe_encode(encoders['area'], area_input)

# Prepare features
features = np.array([[restaurant_type, rate, num_of_ratings, online_order,
                      table_booking, cuisines_type, area, num_cuisines]])

# Predict
predicted_cost = model.predict(features)
print("\n💰 Predicted average cost for two people:", round(predicted_cost[0], 2))


Enter restaurant type:  quick bites
Enter rating (out of 5):  4
Enter number of ratings:  1
Online order available? (1 = Yes, 0 = No):  1
Table booking available? (1 = Yes, 0 = No):  0
Enter cuisine type:  burger
Enter area:  electra city
Enter number of cuisines:  1


⚠️ 'electra city' not seen before. Using default value 'Banashankari' instead.

💰 Predicted average cost for two people: 263.74


C:\Users\amnaj\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [41]:
import pandas as pd

# Define bins and labels
bins = [0, 1000, 2500, float('inf')]
labels = ['Low', 'Medium', 'High']

# Create a new column for class labels
clean_df['cost_category'] = pd.cut(clean_df['avg_cost_(two_people)'], bins=bins, labels=labels)

# Check how it looks
clean_df[['avg_cost_(two_people)', 'cost_category']].head()


,avg_cost_(two_people),cost_category
0,200,Low
1,400,Low
2,400,Low
3,550,Low
4,700,Low


In [42]:

y = clean_df['cost_category']


In [43]:
X = clean_df.drop(['avg_cost_(two_people)', 'cost_category'], axis=1)
y = clean_df['cost_category']


In [44]:
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
y = le_y.fit_transform(y)


In [45]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [57]:
import pandas as pd

# Make sure you have your training feature names
feature_names = X_train.columns

# Wrap the input in a DataFrame with those same column names
input_data = pd.DataFrame(features, columns=feature_names)

# Predict cleanly
predicted_class = model.predict(input_data)


In [59]:
print("Unique labels in y_test:", np.unique(y_test))
print("Unique labels in y_pred:", np.unique(y_pred))

Unique labels in y_test: [0]
Unique labels in y_pred: [0]


In [60]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1278

    accuracy                           1.00      1278
   macro avg       1.00      1.00      1.00      1278
weighted avg       1.00      1.00      1.00      1278


Confusion Matrix:
 [[1278]]


C:\Users\amnaj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [61]:
def safe_encode(encoder, value):
    """Safely encode user input — use default if unseen."""
    if value in encoder.classes_:
        return encoder.transform([value])[0]
    else:
        print(f"⚠️ '{value}' not seen before. Using default value '{encoder.classes_[0]}' instead.")
        return encoder.transform([encoder.classes_[0]])[0]


# Take inputs from user
restaurant_type_input = input("Enter restaurant type: ")
rate = float(input("Enter rating (out of 5): "))
num_of_ratings = int(input("Enter number of ratings: "))
online_order = int(input("Online order available? (1 = Yes, 0 = No): "))
table_booking = int(input("Table booking available? (1 = Yes, 0 = No): "))
cuisines_type_input = input("Enter cuisine type: ")
area_input = input("Enter area: ")
num_cuisines = int(input("Enter number of cuisines: "))

# Encode safely
restaurant_type = safe_encode(encoders['restaurant_type'], restaurant_type_input)
cuisines_type = safe_encode(encoders['cuisines_type'], cuisines_type_input)
area = safe_encode(encoders['area'], area_input)

# Prepare features
features = np.array([[restaurant_type, rate, num_of_ratings, online_order,
                      table_booking, cuisines_type, area, num_cuisines]])

# Predict
predicted_class = model.predict(features)

predicted_label = le_y.inverse_transform(predicted_class)[0]

print("\n🍽️ Predicted Cost Category:", predicted_label)


Enter restaurant type:  fast food
Enter rating (out of 5):  4
Enter number of ratings:  1
Online order available? (1 = Yes, 0 = No):  1
Table booking available? (1 = Yes, 0 = No):  0
Enter cuisine type:  burgers
Enter area:  Kalyan Nagar'
Enter number of cuisines:  1


⚠️ 'fast food' not seen before. Using default value 'bakery' instead.
⚠️ 'burgers' not seen before. Using default value 'african, burger' instead.
⚠️ 'Kalyan Nagar'' not seen before. Using default value 'Banashankari' instead.

🍽️ Predicted Cost Category: Low


C:\Users\amnaj\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [63]:
df.to_csv('cleaned_restaurant_data.csv_model', index=False)
print("\n Cleaned dataset saved as 'cleaned_restaurant_data_model.csv'")


 Cleaned dataset saved as 'cleaned_restaurant_data_model.csv'


In [64]:
model = LogisticRegression(...)  # or LinearRegression
X = df.drop(columns=['avg_cost_(two_people)'])
y = df['avg_cost_(two_people)']


In [69]:
# Create cost ranges manually
df['cost_category'] = pd.cut(
    df['avg_cost_(two_people)'],
    bins=[0, 500, 1500, 5000],  # adjust based on your data range
    labels=['Low', 'Medium', 'High']
)

# Encode labels
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
y = le_y.fit_transform(df['cost_category'])

# Prepare features (drop target + non-numeric)
X = df.drop(columns=['avg_cost_(two_people)', 'cost_category', 'restaurant_name', 'local_address'])

# Train classification model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X, y)

# Predict categories
df['predicted_cost_category'] = le_y.inverse_transform(model.predict(X))


ValueError: could not convert string to float: 'quick bites'